In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import os
import pickle
import boto3
import re
import time

### holy shit
I am writing this on my couch tunnelling through ssh to an EC2 instance, this is the fucking future

In [2]:
iurl = "https://itch.io/games"
turl = "https://itch.io/tags"
new_games_url = "https://itch.io/games/newest"
# https://itch.io/games/newest?page=31 etc
# 105,628 results as of 2018-04-25

In [4]:
# S3 tests

stb = boto3.resource('s3')
for bucket in stb.buckets.all():
    print(bucket.name)

aws-logs-279455525701-us-west-1
derpbot
magonote-html
mascon
pxnx-test
yoon-galvanize-s3
yoon-test-bucket-00


In [5]:
test_data = "urggg! urk urk"
stb.Bucket('pxnx-test').put_object(Key='urg.txt', Body=test_data)

s3.Object(bucket_name='pxnx-test', key='urg.txt')

In [6]:
!echo "blaaaaargh! urg urg" > blerg.urg

# Create an S3 client
s3 = boto3.client('s3')

filename = 'blerg.urg'
bucket_name = 'pxnx-test'

# Uploads the given file using a managed uploader, which will split up large
# files automatically and upload parts in parallel.
s3.upload_file(filename, bucket_name, filename)

In [7]:
!aws s3 ls s3://pxnx-test

                           PRE scans/
                           PRE still/
2018-04-25 20:18:12         20 blerg.urg
2018-04-25 20:18:12         14 urg.txt


In [25]:
def gamelist(url):

    r = requests.get(url)
    rt = r.text
    # print(rt)
    soup = bs(rt, "lxml")
    lclass = "title game_link"
    allanc = soup.find_all("a", class_=lclass)
    # print(allanc)
    lset = set()
    for anc in allanc:
        lset.add(anc.get_text())
    return list(lset)

def taglist(url):
    # note this is for "top tags", not sure what the cutoff is
    r = requests.get(url)
    rt = r.text
    # print(rt)
    soup = bs(rt, "lxml")
    tclass = "outline_button"
    allanc = soup.find_all("a", class_=tclass)
    # print(allanc)
    tset = set()
    for anc in allanc:
        txt = anc.get_text()
        href = anc['href']
        lab = anc['data-label']
        ttup = (txt, href, lab)
        tset.add(ttup)
    return list(tset)

def tagscrape(taglist):
    tbase = "https://itch.io"
    tclist = list()
    for tag in taglist:
        turl = tbase + tag[1]
        r = requests.get(turl)
        rt = r.text
        # print(rt)
        soup = bs(rt, "lxml")
        cclass = "game_count"
        gcount = soup.find("nobr", class_=cclass)
        gctxt = gcount.get_text()
        gcdig = u''.join(c for c in gctxt if '0' <= c <= '9')
        gctup = (tag[0], tag[1], gcdig, tag[2])
        tclist.append(gctup)
    return tclist

def gamelist_from_file(file):
    with open (file, 'r') as fh:
        rt = fh.read()
    soup = bs(rt, "lxml")
    lclass = "title game_link"
    allanc = soup.find_all("a", class_=lclass)
    # print(allanc)
    lset = set()
    for anc in allanc:
        txt = anc.get_text()
        href = anc['href']
        lab = anc['data-label']
        gid = lab.split(':')
        ltup = (txt, href, int(gid[1]))
        lset.add(ltup)
    return list(lset)

def info_from_game_page_file(file):
    with open (file, 'r') as fh:
        rt = fh.read()
    soup = bs(rt, "lxml")
    dclass = "post_content"
    alldiv = soup.find_all("div", class_=dclass)
    # print(allanc)
    lset = set()
    for div in alldiv:
        txt = div.get_text()
        print(txt)
    return list(lset)

[]

In [9]:
# example url: <a class="title game_link" href="https://owch.itch.io/btab" data-label="game:231714:title" data-action="game_grid">BEAT THE ART BREAKER [DEMO]</a>

itchtags = taglist(turl)
# itchlist = gamelist(iurl)

# https://itch.io:443/games/tag-horrible
# https://itch.io:443/games/tag-horrible?page=2


In [10]:
tslist = tagscrape(itchtags[0:6])

In [11]:
tslist

[('Unity', '/games/tag-unity', '2966', 'unity'),
 ('Traps', '/games/tag-traps', '47', 'traps'),
 ('Western', '/games/tag-western', '216', 'western'),
 ('Rhythm', '/games/tag-rhythm', '710', 'rhythm'),
 ('user-interface', '/game-assets/tag-user-interface', '52', 'user-interface'),
 ('Spoopy', '/games/tag-spoopy', '129', 'spoopy')]

In [12]:
# get my curl on
# looks like games end around page 3466 on 2018-04-23
# then 3515 on 2018-04-24
# most pages are around 65000 bytes(?)
# drops off to 26000 when there are no more games
# 105628 games total
# games start at page 1 not zero, just in case you were wondering

# total number of games
total_games = 105628

# start from this page
istart = 1

# don't do it this way
# iend = 2345

# how many pages in each chunk
ichunk = 200

# if you were going to sleep, how long would you sleep for?
izzz = 120

# given a start page and a chunk size, how many reps do you need?
proj_reps = (3600 - istart) / ichunk
print(proj_reps)

17.995


In [13]:
# how many cycles of chunks
irep = 20

In [15]:
# go get all that stuff

for rep in range(irep):
    
    rstart = (rep * ichunk) + istart
    rend = (((rep + 1) * ichunk) + istart) - 1
    srstart = str(rstart)
    srend = str(rend)
    prep = "rsrart: {} rend: {}".format(srstart, srend)
    print(prep)
    # get newest so we don't have to keep doing this
    # new_games_url = "https://itch.io/games/newest?page=321"
    curl = "https://itch.io:443/games/newest?page=[{}-{}]".format(srstart, srend)
    # curl = "https://itch.io:443/games?page=[{}-{}]".format(srstart, srend)
    cfile = "html/itch_game_list_by_newest_{}_{}.html".format(srstart, srend)

    !curl $curl >> $cfile

rsrart: 1 rend: 200

[1/200]: https://itch.io:443/games/newest?page=1 --> <stdout>
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 67705    0 67705    0     0   265k      0 --:--:-- --:--:-- --:--:--  265k

[2/200]: https://itch.io:443/games/newest?page=2 --> <stdout>
100 68886    0 68886    0     0   273k      0 --:--:-- --:--:-- --:--:--  273k

[3/200]: https://itch.io:443/games/newest?page=3 --> <stdout>
100 68722    0 68722    0     0   377k      0 --:--:-- --:--:-- --:--:--  377k

[4/200]: https://itch.io:443/games/newest?page=4 --> <stdout>
100 67304    0 67304    0     0   304k      0 --:--:-- --:--:-- --:--:-- 21.3M

[5/200]: https://itch.io:443/games/newest?page=5 --> <stdout>
100 67571    0 67571    0     0   420k      0 --:--:-- --:--:-- --:--:--  420k

[6/200]: https://itch.io:443/games/newest?page=6 --> <stdout>
100 69382    0 69382    0     0   398k      0 --:

100 68569    0 68569    0     0   367k      0 --:--:-- --:--:-- --:--:-- 50.1M

[58/200]: https://itch.io:443/games/newest?page=58 --> <stdout>
100 69467    0 69467    0     0   318k      0 --:--:-- --:--:-- --:--:--  318k

[59/200]: https://itch.io:443/games/newest?page=59 --> <stdout>
100 69508    0 69508    0     0   336k      0 --:--:-- --:--:-- --:--:--  336k

[60/200]: https://itch.io:443/games/newest?page=60 --> <stdout>
100 69063    0 69063    0     0   312k      0 --:--:-- --:--:-- --:--:--  312k

[61/200]: https://itch.io:443/games/newest?page=61 --> <stdout>
100 68137    0 68137    0     0   408k      0 --:--:-- --:--:-- --:--:--  408k

[62/200]: https://itch.io:443/games/newest?page=62 --> <stdout>
100 69078    0 69078    0     0   399k      0 --:--:-- --:--:-- --:--:-- 50.6M

[63/200]: https://itch.io:443/games/newest?page=63 --> <stdout>
100 69032    0 69032    0     0   343k      0 --:--:-- --:--:-- --:--:--  343k

[64/200]: https://itch.io:443/games/newest?page=64 --> <

100 66729    0 66729    0     0   358k      0 --:--:-- --:--:-- --:--:--  358k

[115/200]: https://itch.io:443/games/newest?page=115 --> <stdout>
100 69569    0 69569    0     0   390k      0 --:--:-- --:--:-- --:--:--  390k

[116/200]: https://itch.io:443/games/newest?page=116 --> <stdout>
100 69272    0 69272    0     0   235k      0 --:--:-- --:--:-- --:--:-- 50.8M

[117/200]: https://itch.io:443/games/newest?page=117 --> <stdout>
100 69522    0 69522    0     0   349k      0 --:--:-- --:--:-- --:--:--  349k

[118/200]: https://itch.io:443/games/newest?page=118 --> <stdout>
100 67302    0 67302    0     0   359k      0 --:--:-- --:--:-- --:--:--  359k

[119/200]: https://itch.io:443/games/newest?page=119 --> <stdout>
100 67781    0 67781    0     0   352k      0 --:--:-- --:--:-- --:--:--  352k

[120/200]: https://itch.io:443/games/newest?page=120 --> <stdout>
100 67437    0 67437    0     0   324k      0 --:--:-- --:--:-- --:--:--  324k

[121/200]: https://itch.io:443/games/newest?

100 68196    0 68196    0     0   317k      0 --:--:-- --:--:-- --:--:-- 49.8M

[172/200]: https://itch.io:443/games/newest?page=172 --> <stdout>
100 69612    0 69612    0     0   331k      0 --:--:-- --:--:-- --:--:--  331k

[173/200]: https://itch.io:443/games/newest?page=173 --> <stdout>
100 69889    0 69889    0     0   383k      0 --:--:-- --:--:-- --:--:--  383k

[174/200]: https://itch.io:443/games/newest?page=174 --> <stdout>
100 69577    0 69577    0     0   390k      0 --:--:-- --:--:-- --:--:--  390k

[175/200]: https://itch.io:443/games/newest?page=175 --> <stdout>
100 66411    0 66411    0     0   360k      0 --:--:-- --:--:-- --:--:--  360k

[176/200]: https://itch.io:443/games/newest?page=176 --> <stdout>
100 68210    0 68210    0     0   406k      0 --:--:-- --:--:-- --:--:--  406k

[177/200]: https://itch.io:443/games/newest?page=177 --> <stdout>
100 67562    0 67562    0     0   227k      0 --:--:-- --:--:-- --:--:-- 49.2M

[178/200]: https://itch.io:443/games/newest?

100 67723    0 67723    0     0   400k      0 --:--:-- --:--:-- --:--:-- 24.6M

[28/200]: https://itch.io:443/games/newest?page=228 --> <stdout>
100 67938    0 67938    0     0   257k      0 --:--:-- --:--:-- --:--:--  257k

[29/200]: https://itch.io:443/games/newest?page=229 --> <stdout>
100 67226    0 67226    0     0   386k      0 --:--:-- --:--:-- --:--:--  386k

[30/200]: https://itch.io:443/games/newest?page=230 --> <stdout>
100 67886    0 67886    0     0   336k      0 --:--:-- --:--:-- --:--:--  336k

[31/200]: https://itch.io:443/games/newest?page=231 --> <stdout>
100 69504    0 69504    0     0   300k      0 --:--:-- --:--:-- --:--:--  300k

[32/200]: https://itch.io:443/games/newest?page=232 --> <stdout>
100 67938    0 67938    0     0   358k      0 --:--:-- --:--:-- --:--:-- 49.5M

[33/200]: https://itch.io:443/games/newest?page=233 --> <stdout>
100 68581    0 68581    0     0   401k      0 --:--:-- --:--:-- --:--:--  401k

[34/200]: https://itch.io:443/games/newest?page=23

...

100 26567    0 26567    0     0   132k      0 --:--:-- --:--:-- --:--:--  132k

[164/200]: https://itch.io:443/games/newest?page=3764 --> <stdout>
100 26566    0 26566    0     0   109k      0 --:--:-- --:--:-- --:--:--  109k

[165/200]: https://itch.io:443/games/newest?page=3765 --> <stdout>
100 26536    0 26536    0     0   130k      0 --:--:-- --:--:-- --:--:--  130k

[166/200]: https://itch.io:443/games/newest?page=3766 --> <stdout>
100 26564    0 26564    0     0   133k      0 --:--:-- --:--:-- --:--:--  133k

[167/200]: https://itch.io:443/games/newest?page=3767 --> <stdout>
100 26575    0 26575    0     0   129k      0 --:--:-- --:--:-- --:--:-- 10.1M

[168/200]: https://itch.io:443/games/newest?page=3768 --> <stdout>
100 26567    0 26567    0     0   133k      0 --:--:-- --:--:-- --:--:--  133k

[169/200]: https://itch.io:443/games/newest?page=3769 --> <stdout>
100 26566    0 26566    0     0   116k      0 --:--:-- --:--:-- --:--:--  116k

[170/200]: https://itch.io:443/games/n

100 26566    0 26566    0     0   120k      0 --:--:-- --:--:-- --:--:--  120k

[19/200]: https://itch.io:443/games/newest?page=3819 --> <stdout>
100 26566    0 26566    0     0   107k      0 --:--:-- --:--:-- --:--:--  107k

[20/200]: https://itch.io:443/games/newest?page=3820 --> <stdout>
100 26575    0 26575    0     0   107k      0 --:--:-- --:--:-- --:--:--  107k

[21/200]: https://itch.io:443/games/newest?page=3821 --> <stdout>
100 26567    0 26567    0     0   116k      0 --:--:-- --:--:-- --:--:-- 12.6M

[22/200]: https://itch.io:443/games/newest?page=3822 --> <stdout>
100 26566    0 26566    0     0   109k      0 --:--:-- --:--:-- --:--:--  109k

[23/200]: https://itch.io:443/games/newest?page=3823 --> <stdout>
100 26567    0 26567    0     0  97672      0 --:--:-- --:--:-- --:--:-- 97672

[24/200]: https://itch.io:443/games/newest?page=3824 --> <stdout>
100 26566    0 26566    0     0   116k      0 --:--:-- --:--:-- --:--:--  116k

[25/200]: https://itch.io:443/games/newest?p

100 26562    0 26562    0     0   112k      0 --:--:-- --:--:-- --:--:--  112k

[76/200]: https://itch.io:443/games/newest?page=3876 --> <stdout>
100 26566    0 26566    0     0   120k      0 --:--:-- --:--:-- --:--:-- 12.6M

[77/200]: https://itch.io:443/games/newest?page=3877 --> <stdout>
100 26564    0 26564    0     0   108k      0 --:--:-- --:--:-- --:--:--  108k

[78/200]: https://itch.io:443/games/newest?page=3878 --> <stdout>
100 26566    0 26566    0     0   107k      0 --:--:-- --:--:-- --:--:--  107k

[79/200]: https://itch.io:443/games/newest?page=3879 --> <stdout>
100 26566    0 26566    0     0   113k      0 --:--:-- --:--:-- --:--:--  113k

[80/200]: https://itch.io:443/games/newest?page=3880 --> <stdout>
100 26539    0 26539    0     0   104k      0 --:--:-- --:--:-- --:--:-- 12.6M

[81/200]: https://itch.io:443/games/newest?page=3881 --> <stdout>
100 26566    0 26566    0     0   116k      0 --:--:-- --:--:-- --:--:--  116k

[82/200]: https://itch.io:443/games/newest?p

100 26575    0 26575    0     0  81769      0 --:--:-- --:--:-- --:--:-- 81769

[132/200]: https://itch.io:443/games/newest?page=3932 --> <stdout>
100 26546    0 26546    0     0   112k      0 --:--:-- --:--:-- --:--:-- 25.3M

[133/200]: https://itch.io:443/games/newest?page=3933 --> <stdout>
100 26567    0 26567    0     0   108k      0 --:--:-- --:--:-- --:--:--  108k

[134/200]: https://itch.io:443/games/newest?page=3934 --> <stdout>
100 26567    0 26567    0     0   125k      0 --:--:-- --:--:-- --:--:--  125k

[135/200]: https://itch.io:443/games/newest?page=3935 --> <stdout>
100 26566    0 26566    0     0   139k      0 --:--:-- --:--:-- --:--:--  139k

[136/200]: https://itch.io:443/games/newest?page=3936 --> <stdout>
100 26564    0 26564    0     0   116k      0 --:--:-- --:--:-- --:--:--  116k

[137/200]: https://itch.io:443/games/newest?page=3937 --> <stdout>
100 26564    0 26564    0     0   116k      0 --:--:-- --:--:-- --:--:-- 25.3M

[138/200]: https://itch.io:443/games/n

100 26566    0 26566    0     0  97669      0 --:--:-- --:--:-- --:--:-- 97669

[188/200]: https://itch.io:443/games/newest?page=3988 --> <stdout>
100 26566    0 26566    0     0  97311      0 --:--:-- --:--:-- --:--:-- 97311

[189/200]: https://itch.io:443/games/newest?page=3989 --> <stdout>
100 26541    0 26541    0     0   112k      0 --:--:-- --:--:-- --:--:--  112k

[190/200]: https://itch.io:443/games/newest?page=3990 --> <stdout>
100 26564    0 26564    0     0   109k      0 --:--:-- --:--:-- --:--:-- 25.3M

[191/200]: https://itch.io:443/games/newest?page=3991 --> <stdout>
100 26566    0 26566    0     0  83018      0 --:--:-- --:--:-- --:--:-- 83018

[192/200]: https://itch.io:443/games/newest?page=3992 --> <stdout>
100 26566    0 26566    0     0   101k      0 --:--:-- --:--:-- --:--:--  101k

[193/200]: https://itch.io:443/games/newest?page=3993 --> <stdout>
100 26575    0 26575    0     0   111k      0 --:--:-- --:--:-- --:--:--  111k

[194/200]: https://itch.io:443/games/n

In [14]:
htmdir = "html"

htmlist = list()
for file in os.listdir(htmdir):
    if file[-5:] == '.html':
        htmlist.append(file)

bilist = list()
for itchfile in htmlist:
    fpath = "html/" + itchfile
    sublist = gamelist_from_file(fpath)
    bilist.extend(sublist)
    
len(bilist)

105625

In [17]:
with open('all_games.pkl','wb') as fh:
    pickle.dump(bilist,fh)

In [15]:
bilist[546]

('Singed Dirt', 'https://lanthos.itch.io/singed-dirt', 146499)

In [26]:
# ok dude you really ought to put a progress bar or counter or something in here
# ran for ummmm hmmm ok well let's see...

# ubuntu@ip-172-30-0-129:~/galvanize-magonote/html$ ls -lt game_pages | grep -v '^d' | tail -1
# -rw-rw-r-- 1 ubuntu ubuntu  16108 Apr 24 21:39 111303.html

# ubuntu@ip-172-30-0-129:~/galvanize-magonote/html$ ls -1 game_pages | wc -l
# 24675

# ubuntu@ip-172-30-0-129:~/galvanize-magonote/html$ date
# Wed Apr 25 20:02:11 UTC 2018

# so uh 24675 files in 20:02 - 21:39 = 1h37m or 97m
# about 250 files per minute...
# so yeah maybe don't run this in the middle of the day when you're doing other stuff

game_page_file_list = list()

for item in bilist[20220:20228]:
    curl = item[1]
    cfile = "html/game_pages/{}.html".format(item[2])
    
    game_page_file_list.append(cfile)
    !curl $curl > $cfile
        

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 27289    0 27289    0     0   306k      0 --:--:-- --:--:-- --:--:--  306k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17992    0 17992    0     0   183k      0 --:--:-- --:--:-- --:--:--  183k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22087    0 22087    0     0   326k      0 --:--:-- --:--:-- --:--:--  326k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16765    0 16765    0     0   282k      0 --:--:-- --:--:-- --:--:--  282k
  % Total    % Received % Xferd  Average Speed   Tim

In [27]:
for game_file in game_page_file_list:
    print(info_from_game_page_file(game_file))

thedudeist323 days ago(+1)Fun game, spent a good hour on creating a nice 20 edged(lol) sword that did wonders in damage. unfortunately i ran into the bug where you get stuck in the center of the area on one of the platforms. Then i had to close, and unfortunately all my effort went to waste when i realized the save function only worked in a single sitting and didn't extend to a second session. Very frustrated.  But still, the crafting system is unique and very fun.Reply
Lucas Demo322 days agoThanks for the commentary :)So, as you said - the game right now has a lot of problems. Some of them you pointed, some of them people dont notice but they do exist. The game need to be fixed is all kind of aspects, and that is probably ALL reflex of the fact that it has been created in fully made in 48 hours. I usually do not change anything after the ludumdare, but SMASHSMITH is right now in a strange position, it's representing 95% of my downloads, what makes me think about fixing it. Probably yo

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5723    0  5723    0     0   186k      0 --:--:-- --:--:-- --:--:--  186k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20641    0 20641    0     0   314k      0 --:--:-- --:--:-- --:--:--  314k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7928    0  7928    0     0   151k      0 --:--:-- --:--:-- --:--:--  151k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14379    0 14379    0     0   237k      0 --:--:-- --:--:-- --:--:--  237k
  % Total    % Received % Xferd  Average Speed   Tim

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8869    0  8869    0     0   129k      0 --:--:-- --:--:-- --:--:--  129k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5721    0  5721    0     0   164k      0 --:--:-- --:--:-- --:--:--  164k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7631    0  7631    0     0   162k      0 --:--:-- --:--:-- --:--:--  162k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8452    0  8452    0     0   116k      0 --:--:-- --:--:-- --:--:--  116k
  % Total    % Received % Xferd  Average Speed   Tim

100  9846    0  9846    0     0   200k      0 --:--:-- --:--:-- --:--:--  200k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8010    0  8010    0     0   205k      0 --:--:-- --:--:-- --:--:--  205k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7709    0  7709    0     0   156k      0 --:--:-- --:--:-- --:--:--  156k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8790    0  8790    0     0   238k      0 --:--:-- --:--:-- --:--:--  238k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9158    0  9158    0     0   223k      0 --:--:

100  8304    0  8304    0     0   144k      0 --:--:-- --:--:-- --:--:--  147k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5723    0  5723    0     0   139k      0 --:--:-- --:--:-- --:--:--  139k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5723    0  5723    0     0   192k      0 --:--:-- --:--:-- --:--:--  192k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8254    0  8254    0     0   155k      0 --:--:-- --:--:-- --:--:--  155k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5723    0  5723    0     0   164k      0 --:--:

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8369    0  8369    0     0   194k      0 --:--:-- --:--:-- --:--:--  190k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5723    0  5723    0     0   133k      0 --:--:-- --:--:-- --:--:--  136k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7965    0  7965    0     0   189k      0 --:--:-- --:--:-- --:--:--  189k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5723    0  5723    0     0   174k      0 --:--:-- --:--:-- --:--:--  174k
  % Total    % Received % Xferd  Average Speed   Tim

...

100 11114    0 11114    0     0   217k      0 --:--:-- --:--:-- --:--:--  217k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8194    0  8194    0     0   228k      0 --:--:-- --:--:-- --:--:--  228k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5723    0  5723    0     0   186k      0 --:--:-- --:--:-- --:--:--  186k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11590    0 11590    0     0   297k      0 --:--:-- --:--:-- --:--:--  297k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8032    0  8032    0     0   174k      0 --:--:

100  8081    0  8081    0     0   154k      0 --:--:-- --:--:-- --:--:--  154k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8042    0  8042    0     0   178k      0 --:--:-- --:--:-- --:--:--  178k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5723    0  5723    0     0   186k      0 --:--:-- --:--:-- --:--:--  186k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8392    0  8392    0     0   186k      0 --:--:-- --:--:-- --:--:--  186k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11650    0 11650    0     0   232k      0 --:--:

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7838    0  7838    0     0   201k      0 --:--:-- --:--:-- --:--:--  201k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5723    0  5723    0     0   192k      0 --:--:-- --:--:-- --:--:--  192k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5723    0  5723    0     0   180k      0 --:--:-- --:--:-- --:--:--  180k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8674    0  8674    0     0   116k      0 --:--:-- --:--:-- --:--:--  116k
  % Total    % Received % Xferd  Average Speed   Tim

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8276    0  8276    0     0   164k      0 --:--:-- --:--:-- --:--:--  164k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5723    0  5723    0     0   155k      0 --:--:-- --:--:-- --:--:--  155k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5723    0  5723    0     0   186k      0 --:--:-- --:--:-- --:--:--  186k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8462    0  8462    0     0   121k      0 --:--:-- --:--:-- --:--:--  121k
  % Total    % Received % Xferd  Average Speed   Tim

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7081    0  7081    0     0   177k      0 --:--:-- --:--:-- --:--:--  177k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5723    0  5723    0     0   180k      0 --:--:-- --:--:-- --:--:--  180k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6707    0  6707    0     0  95814      0 --:--:-- --:--:-- --:--:-- 95814
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8218    0  8218    0     0   205k      0 --:--:-- --:--:-- --:--:--  205k
  % Total    % Received % Xferd  Average Speed   Tim

In [61]:
# ok time to break this guy out. stand by please

def info_from_game_post_file(file):
    pid_l = re.findall(r'\d+', file)
    pid = pid_l[0]
    did = 'post-' + pid
    # print(did)
    with open (file, 'r') as fh:
        rt = fh.read()
    soup = bs(rt, "lxml")
    dclass = "post_content"
    div = soup.find('div', id=did)
    dtag = soup.div
    sdiv = dtag.attrs
    print(sdiv)

In [62]:
print(info_from_game_post_file('html/post_pages/435.html'))

{'id': 'header_widget_61163', 'class': ['header_widget']}
None


In [17]:
columns = ['name', 'url', 'gid']

df = pd.DataFrame([g for g in bilist], columns=columns)

# print df

df = df.set_index('gid')
#if you need set column name
df.columns.name = 'game_id'

df.sort_index()

game_id,name,url
gid,,
3,X-Moon,https://leafo.itch.io/x-moon
16,Escape From Andromeda,https://adit.itch.io/escape-from-andromeda
24,DARKLORD - An Arcade Platformer,https://tyoung.itch.io/darklord---an-arcade-pl...
28,Homunculus,https://doom-crow.itch.io/homunculus
37,PEA,https://gameofmax.itch.io/pea
39,Let's Learn Japanese: Deluxe,https://gzstorm.itch.io/lets-learn-japanese-de...
41,Adlib,https://spottedzebra.itch.io/adlib
42,Planet's Core,https://2bitent.itch.io/planetscore
44,Queen Frances and the Rat Rebellion,https://manikin.itch.io/rat-rebellion


In [18]:
df['url']

gid
176389                  https://egn-games.itch.io/run-block
240350                     https://mavw.itch.io/aeroproject
194540              https://link990.itch.io/orb-of-the-cave
188791    https://stentann.itch.io/phoenixgaming2dplatfo...
16032              https://letmethink.itch.io/slimeys-quest
32379             https://herocraft.itch.io/catch-the-candy
113493                  https://strdr4605.itch.io/wave-hero
193144                  https://ury18.itch.io/shark-rampage
163476      https://viashimo.itch.io/ld39-dead-in-the-water
14188                         https://raon.itch.io/the-cave
203393                 https://bobbybubi.itch.io/prep-story
199540    https://geneticastudios.itch.io/eljardindelafe...
225133                https://saagamestudio.itch.io/zomboid
163117     https://ministerjay.itch.io/wobbling-in-the-dark
164755                    https://radiosoap.itch.io/nothing
218325                  https://kandersen.itch.io/boundless
113576            https://becklespin

In [17]:
biglist[12]

('Kingdom Hearts nobody survives. (3d)',
 'https://horrornerd.itch.io/kingdom-hearts-nobody-survives',
 73957)

In [4]:
# this adds a little nap between downloads, looks like that prevents itch's server
# from giving us garbage

# find post_pages/ -size -1000c -type f -delete
# will find all files in a dir smaller than 1000 bytes

# ls -1 post_pages | wc -l
# will count files in a dir

ppdir = "html/post_pages"
test_count = 0
dl_count = 0

post_list = list()
post_set = set(os.listdir(ppdir))

for post in range(2010):
    test_str = str(post) + ".html"
    if test_str in post_set:
        test_count += 1
    else:
        dl_count += 1
        purl = "https://itch.io:443/post/" + str(post)
        dlfile = ppdir + "/" + test_str
        response = requests.get(purl)
        time.sleep(1)
        rt = response.text
        with open(dlfile, "w") as fh:
            fh.write(rt)

    
print(test_count, dl_count)


2010 0


In [68]:
post_set_list = list(post_set)

In [70]:
post_set_list[234]

'202857.html'